In [50]:
from itertools import chain
from lnn import Predicate, Variable, Exists, Implies, And, ForAll, Model, Fact, World

## Domain specification

In [51]:
# entities and properties names
entities = ["rectangle", "square", "rhombus", "parallelogram"]
properties = ["foursided", "equal_sides", "right_angles"]

# implications
entity_relationships = [
    ("square", "rectangle"),
    ("square", "rhombus"),
    ("rectangle", "parallelogram"),
    ("rhombus", "parallelogram"),
]

# implications
entity_properties = [
    ("parallelogram", "foursided"),
    ("rectangle", "right_angles"),
    ("rhombus", "equal_sides"),
]

# information that object "s1" is a square etc.
facts = {
    "square": ["s1", "s2"],
    "rectangle": ["r1"],
    "rhombus": ["rh1"],
    "parallelogram": ["p1"],
}

## Model definition and domain parsing

In [63]:
model = Model()

x = Variable("x")

# parsing predicates
for predicate in chain(entities, properties):
    model[predicate] = Predicate(name=predicate)

model.nodes

{'rectangle': <lnn.symbolic.logic.Predicate at 0x111ce7c70>,
 'square': <lnn.symbolic.logic.Predicate at 0x111b12e20>,
 'rhombus': <lnn.symbolic.logic.Predicate at 0x111b26ac0>,
 'parallelogram': <lnn.symbolic.logic.Predicate at 0x111ce7700>,
 'foursided': <lnn.symbolic.logic.Predicate at 0x111ce7b80>,
 'equal_sides': <lnn.symbolic.logic.Predicate at 0x111ce7520>,
 'right_angles': <lnn.symbolic.logic.Predicate at 0x111cfdc10>}

In [64]:
# parsing axioms in Implication and ForAll shape
axioms = [
    ForAll(
        x,
        Implies(model[p1](x), model[p2](x), name=f"{p1}_{p2}"),
        name=f"all_{p1}_{p2}",
        world=World.AXIOM,
    )
    for p1, p2 in chain(entity_relationships, entity_properties)
]

model.add_formulae(*axioms)

model.nodes

{'rectangle': <lnn.symbolic.logic.Predicate at 0x111ce7c70>,
 'square': <lnn.symbolic.logic.Predicate at 0x111b12e20>,
 'rhombus': <lnn.symbolic.logic.Predicate at 0x111b26ac0>,
 'parallelogram': <lnn.symbolic.logic.Predicate at 0x111ce7700>,
 'foursided': <lnn.symbolic.logic.Predicate at 0x111ce7b80>,
 'equal_sides': <lnn.symbolic.logic.Predicate at 0x111ce7520>,
 'right_angles': <lnn.symbolic.logic.Predicate at 0x111cfdc10>,
 'all_square_rectangle': <lnn.symbolic.logic.ForAll at 0x111d173a0>,
 'square_rectangle': <lnn.symbolic.logic.Implies at 0x111da73d0>,
 'all_square_rhombus': <lnn.symbolic.logic.ForAll at 0x111da7430>,
 'square_rhombus': <lnn.symbolic.logic.Implies at 0x111d17190>,
 'all_rectangle_parallelogram': <lnn.symbolic.logic.ForAll at 0x111b12dc0>,
 'rectangle_parallelogram': <lnn.symbolic.logic.Implies at 0x111da7460>,
 'all_rhombus_parallelogram': <lnn.symbolic.logic.ForAll at 0x111db6280>,
 'rhombus_parallelogram': <lnn.symbolic.logic.Implies at 0x111d1fc10>,
 'all_par

In [65]:
# parsing facts
model.add_facts(
    {entity: {name: Fact.TRUE for name in names} for entity, names in facts.items()}
)

## Question definition

In [66]:
queries = [
    Exists(x, model["foursided"](x), name="foursided_objects"),
    Exists(x, model["equal_sides"](x), name="equal_sides_objects"),
    Exists(x, model["right_angles"](x), name="right_angles_objects"),
    Exists(
        x,
        And(model["right_angles"](x), model["equal_sides"](x)),
        name="right_angles_and_equal_length_sides",
    ),
]

model.add_formulae(*queries)

model.nodes

{'rectangle': <lnn.symbolic.logic.Predicate at 0x111ce7c70>,
 'square': <lnn.symbolic.logic.Predicate at 0x111b12e20>,
 'rhombus': <lnn.symbolic.logic.Predicate at 0x111b26ac0>,
 'parallelogram': <lnn.symbolic.logic.Predicate at 0x111ce7700>,
 'foursided': <lnn.symbolic.logic.Predicate at 0x111ce7b80>,
 'equal_sides': <lnn.symbolic.logic.Predicate at 0x111ce7520>,
 'right_angles': <lnn.symbolic.logic.Predicate at 0x111cfdc10>,
 'all_square_rectangle': <lnn.symbolic.logic.ForAll at 0x111d173a0>,
 'square_rectangle': <lnn.symbolic.logic.Implies at 0x111da73d0>,
 'all_square_rhombus': <lnn.symbolic.logic.ForAll at 0x111da7430>,
 'square_rhombus': <lnn.symbolic.logic.Implies at 0x111d17190>,
 'all_rectangle_parallelogram': <lnn.symbolic.logic.ForAll at 0x111b12dc0>,
 'rectangle_parallelogram': <lnn.symbolic.logic.Implies at 0x111da7460>,
 'all_rhombus_parallelogram': <lnn.symbolic.logic.ForAll at 0x111db6280>,
 'rhombus_parallelogram': <lnn.symbolic.logic.Implies at 0x111d1fc10>,
 'all_par

## Infer from model and answer questions

In [67]:
steps, facts_inferred = model.infer()

for query in queries:
    print(f"Query {query.name} result:")
    print(model[query.name].true_groundings)

Query foursided_objects result:
{'p1', 'r1', 'rh1', 's1', 's2'}
Query equal_sides_objects result:
{'s2', 'rh1', 's1'}
Query right_angles_objects result:
{'s2', 'r1', 's1'}
Query right_angles_and_equal_length_sides result:
{'s2', 's1'}
